<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install ktrain

In [15]:
import os
import sys
import wget
import nltk
import ktrain
import tarfile
import warnings
import numpy as np
from ktrain import text
import tensorflow as tf
from zipfile import ZipFile

warnings.filterwarnings('ignore')

# Data
###IMDB Dataset

In [ ]:
!wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
!unzip DATAPATH/glove.6B.zip -d DATAPATH/glove.6B

!wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH
BASE_DIR = 'Data'

In [4]:
BASE_DIR = '/content/DATAPATH'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [5]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [6]:
def get_data(data_dir):
  texts = []
  labels_index = {'pos': 1, 'neg': 0}
  labels = []
  for name in sorted(os.listdir(data_dir)):
    path = os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id = labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath = os.path.join(path, fname)
          text = open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)
  return texts, labels

In [7]:
train_texts, train_labels = get_data(TRAIN_DATA_DIR)
test_texts, test_labels = get_data(TEST_DATA_DIR)

In [8]:
train_texts[20]

"The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil."

#DNN

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index

In [16]:
train_valid_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_valid_labels = tf.keras.utils.to_categorical(np.asarray(train_labels))
test_labels = tf.keras.utils.to_categorical(np.asarray(test_labels))

In [17]:
indices =np.arange(train_valid_data.shape[0])
np.random.shuffle(indices)
train_valid_data = train_valid_data[indices]
train_valid_labels = train_valid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * train_valid_data.shape[0])

In [18]:
x_train = train_valid_data[:-num_validation_samples]
y_train = train_valid_labels[:-num_validation_samples]

x_train = train_valid_data[-num_validation_samples:]
y_train = train_valid_labels[-num_validation_samples:]

In [20]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

#BERT Sentiment Classification
## IMDB ktrain